In [1]:
from collections import deque
from typing import List, Tuple

from pydantic import BaseModel, Field
from spondee.search import nlp_pipeline, identify_statements, sentence_metadata

In [2]:
nlp = nlp_pipeline()

2024-07-07 01:28:45 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-07-07 01:28:45 INFO: Downloaded file to /Users/tylerbrown/stanza_resources/resources.json
2024-07-07 01:28:45 WARNING: Language en package default expects mwt, which has been added
2024-07-07 01:28:45 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

2024-07-07 01:28:45 INFO: Using device: cpu
2024-07-07 01:28:45 INFO: Loading: tokenize
2024-07-07 01:28:46 INFO: Loading: mwt
2024-07-07 01:28:46 INFO: Loading: pos
2024-07-07 01:28:46 INFO: Loading: constituency
2024-07-07 01:28:46 INFO: Done loading processors!


In [3]:
txt = "".join(
        [
            "Gavin Sheets hit his first career grand slam, and the ",
            "Chicago White Sox won their second straight after a ",
            "franchise-record 14-game losing streak, beating the ",
            "Boston Red Sox 6-1 on Saturday.",
        ]
    )

In [8]:
doc = nlp(txt)